In [102]:
import pandas as pd
import os
import rasterio
import numpy as np
from datetime import datetime, time

In [77]:
# Ruta al archivo CSV
archivo_csv = r'C:\Users\1\Desktop\datos_grafica\Coordenadas_Grosseto.csv'

# Cargar el archivo CSV en un DataFrame de pandas con i parametri delimitatori corretti
coord_df = pd.read_csv(archivo_csv, delimiter=';', decimal=',')
coord_df.rename(columns={'longitudine': 'lon', 'latitudine': 'lat'}, inplace=True)


# Mostrar el DataFrame
print(coord_df)

   Coordenadas puntos        lon        lat
0     Black plastic 1  11.067751  42.860612
1              Corn 1  11.067724  42.850206
2    Clover station 1  11.065604  42.854430
3     Clover (high) 1  11.076013  42.830474
4      Clover pivot 1  11.070988  42.847443
5             Water 1  11.071573  42.851261
6     Black plastic 2  11.067660  42.860737
7              Corn 2  11.067744  42.850149
8    Clover station 2  11.065461  42.854781
9     Clover (high) 2  11.076012  42.830474
10     Clover pivot 2  11.071106  42.847634
11            Water 2  11.071010  42.851510
12    Black plastic 3  11.067728  42.860673
13             Corn 3  11.067704  42.850273
14   Clover station 3  11.065391  42.855859
15    Clover (high) 3  11.076013  42.830470
16     Clover pivot 3  11.071154  42.847523
17            Water 3  11.071120  42.851660


In [79]:
# Percorso alla cartella delle immagini
cartella_imagenes = r'C:\Users\1\Desktop\datos_grafica\Imagenes_LST_grosseto_gday'

# Ottenere una lista di tutti i nomi dei file nella cartella
nomi_immagini = os.listdir(cartella_imagenes)
print(nomi_immagini)

# Creare una lista con i nomi delle colonne
nomi_colonne = ['Tipo Superficie', 'Nome Immagine', 'Valore LST']

# Ottenere il numero di immagini analizzate
numero_immagini_analizzate = len(nomi_immagini)
print(f"Numero di immagini analizzate: {numero_immagini_analizzate}")

# Ottenere il numero di punti delle coordinate prima dell'iterazione
numero_punti_coordinate_prima = len(coord_df)
print(f"Numero di punti delle coordinate prima dell'iterazione: {numero_punti_coordinate_prima}")

# Creare un elenco di dizionari per immagini e punti delle coordinate
righe_da_aggiungere = []

# Iterare su ogni immagine
for nome_immagine in nomi_immagini:
    # Estrarre l'ora dal nome dell'immagine
    ora_str = nome_immagine.split('_')[1].replace('.tif', '')
    # Convertire l'ora in un formato desiderato
    ora = pd.to_datetime(ora_str, format='%H%M%S').strftime('%H:%M:%S')

    # Aprire l'immagine raster
    percorso_immagine = os.path.join(cartella_imagenes, nome_immagine)
    with rasterio.open(percorso_immagine) as src:
        # Iterare su ogni punto nel file CSV
        for index, row in coord_df.iterrows():
            # Ottenere le coordinate del punto
            lon, lat = row['lon'], row['lat']
            
            # Convertire le coordinate in coordinate di pixel
            x, y = src.index(lon, lat)

            # Verificare se le coordinate sono all'interno dei limiti dell'immagine
            if 0 <= x < src.width and 0 <= y < src.height:
                # Leggere il valore LST dal raster
                valore_lst = src.read(1, window=((y, y+1), (x, x+1)))

                # Ottenere il tipo di superficie dal file CSV
                tipo_superficie = row['Coordenadas puntos']

            # Aggiungere la riga all'elenco di dizionari
            riga_da_aggiungere = {'Tipo Superficie': tipo_superficie, 'Ora': ora, 'Valore LST': valore_lst[0]}
            righe_da_aggiungere.append(riga_da_aggiungere)

    # Creare il DataFrame una sola volta
    df = pd.DataFrame(righe_da_aggiungere)
                
                # Mostrare il DataFrame risultante
#print(df)

# Creare un nuovo DataFrame con solo le righe corrette
df_corretto = pd.DataFrame(columns=nomi_colonne)

# Creare un set per tenere traccia dei punti già considerati per ogni immagine
punti_già_aggiunti_per_immagine = set()

# Filtrare le righe corrette
for index, row in df.iterrows():
    # Verificare se i valori di LST sono vicini a -3.4e+38
    if not np.allclose(row['Valore LST'], -3.4e+38):
        # Creare una chiave univoca per identificare l'immagine e il punto
        chiave = (row['Tipo Superficie'], row['Ora'])

        # Verificare se il punto è già stato considerato per questa immagine
        if chiave not in punti_già_aggiunti_per_immagine:
            # Aggiungere la riga al DataFrame corretto
            df_corretto = pd.concat([df_corretto, pd.DataFrame([row])], ignore_index=True)

            # Aggiungere il punto al set dei punti già considerati per questa immagine
            punti_già_aggiunti_per_immagine.add(chiave)

# Mostrare il DataFrame risultante corretto
print(df_corretto)

# Percorso completo per il salvataggio del file CSV
percorso_salvataggio = r'C:\Users\1\Desktop\datos_grafica\Tabla_imagenes_LST_3puntos.csv'

# Salvare il DataFrame come file CSV
# df_corretto.to_csv(percorso_salvataggio, index=False)

print(f"Il file CSV è stato salvato con successo in: {percorso_salvataggio}")


['LST_074815.tif', 'LST_080548.tif', 'LST_081050.tif', 'LST_081632.tif', 'LST_082149.tif', 'LST_082715.tif', 'LST_083210.tif', 'LST_083742.tif', 'LST_084221.tif', 'LST_084719.tif', 'LST_085221.tif', 'LST_085723.tif', 'LST_090225.tif', 'LST_090737.tif', 'LST_091243.tif', 'LST_091727.tif', 'LST_092505.tif', 'LST_093414.tif', 'LST_093921.tif', 'LST_094718.tif', 'LST_095206.tif', 'LST_095711.tif', 'LST_105340.tif', 'LST_110711.tif', 'LST_111848.tif', 'LST_112619.tif', 'LST_113948.tif', 'LST_114528.tif', 'LST_114947.tif', 'LST_115402.tif', 'LST_115837.tif', 'LST_120305.tif', 'LST_120735.tif', 'LST_121220.tif', 'LST_121645.tif', 'LST_122121.tif', 'LST_122535.tif', 'LST_123008.tif', 'LST_123515.tif', 'LST_124001.tif', 'LST_124451.tif', 'LST_125437.tif', 'LST_125956.tif', 'LST_130441.tif', 'LST_130830.tif', 'LST_131257.tif', 'LST_131726.tif', 'LST_132234.tif', 'LST_132651.tif', 'LST_133117.tif', 'LST_133526.tif', 'LST_133958.tif', 'LST_134426.tif', 'LST_134846.tif', 'LST_135301.tif', 'LST_1358

In [106]:
#Open the excel file with in situ data
df_apogee = pd.read_excel(r'C:\Users\1\Desktop\datos_grafica\datos_apogee_cimel.xlsx', sheet_name='APOGEE')
# print(df_apogee)

df_cimel = pd.read_excel(r'C:\Users\1\Desktop\datos_grafica\datos_apogee_cimel.xlsx', sheet_name='CIMEL')
# print(df_cimel)

# Aggiungi la data solo alle prime 185 righe
df_apogee.loc[:184, 'Time (UTC)'] = df_apogee.loc[:184, 'Time (UTC)'].apply(lambda x: datetime.combine(datetime(2023, 6, 26), x))

# Mostra il DataFrame risultante
print(df_apogee)


               Time (UTC) Field Type  Apogee T (K)
0     2023-06-26 07:41:00      Water    299.450000
1     2023-06-26 07:41:00      Water    299.550000
2     2023-06-26 07:41:00      Water    299.950000
3     2023-06-26 07:41:00      Water    299.950000
4     2023-06-26 07:41:00      Water    299.950000
...                   ...        ...           ...
1256  2023-06-26 11:42:00       Corn    306.065586
1257  2023-06-26 11:44:00       Corn    305.864372
1258  2023-06-26 11:46:00       Corn    305.885558
1259  2023-06-26 11:48:00       Corn    306.763599
1260  2023-06-26 11:50:00       Corn    306.858665

[1261 rows x 3 columns]


In [118]:
# Compare the data - Apogee and LST images
print(df_corretto) 

# 1. Assicurati che le colonne abbiano lo stesso tipo di dato
df_corretto['Time (UTC)'] = pd.to_datetime(df_corretto['Time (UTC)'])
df_apogee['Time (UTC)'] = pd.to_datetime(df_apogee['Time (UTC)'], errors='coerce')

print(df_corretto) 

# 2. Creare una funzione per mappare i tipi di superficie
def mappa_tipi_superficie(tipo_superficie):
    if 'water' in tipo_superficie.lower():
        return 'Water'
    elif 'corn' in tipo_superficie.lower():
        return 'Corn'
    elif 'clover (high)' in tipo_superficie.lower():
        return 'Clover (high)'
    elif 'black plastic' in tipo_superficie.lower():
        return 'Black Plastic'
    # Aggiungi altri casi secondo necessità
    else:
        return tipo_superficie

# 3. Creare una colonna nel DataFrame df_corretto con i tipi di superficie mappati
df_corretto['Tipo Superficie Mappato'] = df_corretto['Tipo Superficie'].apply(mappa_tipi_superficie)

# Elimina le colonne 'Tipo Superficie' e 'Nome Immagine'
df_corretto.drop(['Tipo Superficie', 'Nome Immagine'], axis=1, inplace=True)

# Rinomina la colonna 'Ora' in 'Time (UTC)'
df_corretto.rename(columns={'Ora': 'Time (UTC)'}, inplace=True)
df_corretto.rename(columns={'Valore LST': 'T (K)'}, inplace=True)

# Mostra il DataFrame risultante
print(df_corretto)

# 4. Rinomina la colonna 'Field Type' in 'Tipo Superficie Mappato' nel DataFrame df_apogee
df_apogee.rename(columns={'Field Type': 'Tipo Superficie Mappato'}, inplace=True)
df_apogee.rename(columns={'Apogee T (K)': 'T (K)'}, inplace=True)

print(df_apogee)

# 5. Unire i due DataFrame in base alle colonne dei tipi di superficie mappati e alle differenze di ora desiderate
differenza_ora_max = pd.Timedelta(minutes=15)

# Ordina i DataFrame prima di eseguire la fusione
df_corretto = df_corretto.sort_values(by=['Tipo Superficie Mappato', 'Ora'])
df_apogee = df_apogee.sort_values(by=['Tipo Superficie Mappato', 'Time (UTC)'])

# Esegui la fusione
# merged_df = pd.merge_asof(df_corretto, df_apogee, by='Tipo Superficie Mappato', left_on='Ora', right_on='Time (UTC)', tolerance=differenza_ora_max)

# Mostra i risultati
print(merged_df)




           T (K)          Time (UTC) Tipo Superficie Mappato
51    [322.7837] 2024-01-30 09:02:25           Black Plastic
52   [320.54865] 2024-01-30 09:02:25           Black Plastic
53   [319.44574] 2024-01-30 09:02:25           Black Plastic
103  [315.82153] 2024-01-30 12:12:20           Black Plastic
104  [319.14267] 2024-01-30 12:12:20           Black Plastic
..           ...                 ...                     ...
151   [311.0772] 2024-01-30 12:44:51                   Water
153  [313.74805] 2024-01-30 12:44:51                   Water
158  [318.15195] 2024-01-30 13:04:41                   Water
160   [317.6031] 2024-01-30 13:04:41                   Water
162  [317.05603] 2024-01-30 13:04:41                   Water

[199 rows x 3 columns]
           T (K)          Time (UTC) Tipo Superficie Mappato
51    [322.7837] 2024-01-30 09:02:25           Black Plastic
52   [320.54865] 2024-01-30 09:02:25           Black Plastic
53   [319.44574] 2024-01-30 09:02:25           Black Plastic


KeyError: 'Tipo Superficie'